In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv("homework_6.1.csv")
df.head(5)

,Unnamed: 0,Z,X,Y
0,0,0.548814,0,-0.823220
1,1,0.715189,1,0.842405
2,2,0.602763,1,0.898618
3,3,0.544883,0,-0.817325
4,4,0.423655,0,-0.635482


In [3]:
df = df.drop(columns=['Unnamed: 0'])

treated = df[df['X'] == 1].reset_index(drop=True)
untreated = df[df['X'] == 0].reset_index(drop=True)

nn = NearestNeighbors(n_neighbors=1)
nn.fit(untreated[['Z']])

distances, indices = nn.kneighbors(treated[['Z']])
treated_counterfactual_Y = untreated.loc[indices[:,0], 'Y'].values
treated_Y = treated['Y'].values

treatment_effects_treated = treated_Y - treated_counterfactual_Y

att = np.mean(treatment_effects_treated)

nn.fit(treated[['Z']])
distances_u, indices_u = nn.kneighbors(untreated[['Z']])
untreated_counterfactual_Y = treated.loc[indices_u[:,0], 'Y'].values
untreated_Y = untreated['Y'].values

treatment_effects_untreated = untreated_counterfactual_Y - untreated_Y

atu = np.mean(treatment_effects_untreated)

ate = np.mean(np.concatenate([treatment_effects_treated, treatment_effects_untreated]))


mte = np.max(treatment_effects_untreated)


print(f"Average Treatment Effect (ATE): {ate:.4f}")
print(f"Average Treatment Effect on the Treated (ATT): {att:.4f}")
print(f"Average Treatment Effect on the Untreated (ATU): {atu:.4f}")
print(f"Marginal Treatment Effect (MTE): {mte:.4f}")

Average Treatment Effect (ATE): 1.6953
Average Treatment Effect on the Treated (ATT): 1.8464
Average Treatment Effect on the Untreated (ATU): 1.5495
Marginal Treatment Effect (MTE): 2.1725


In [4]:
df = pd.read_csv("homework_6.1.csv")
treated = df[df['X'] == 1].reset_index(drop=True)
untreated = df[df['X'] == 0].reset_index(drop=True)
# Match untreated to treated using Z
Z_untreated = untreated[['Z']]
Z_treated = treated[['Z']]
nn = NearestNeighbors(n_neighbors=1).fit(Z_treated)
_, idx = nn.kneighbors(Z_untreated)

# Compute treatment effects
Y_untreated = untreated['Y'].values
Y_matched_treated = treated.loc[idx.flatten(), 'Y'].values
effects = Y_matched_treated - Y_untreated

# Use top 5% of effects
top_5_count = max(1, int(len(effects) * 0.05))
top_5_effects = np.sort(effects)[-top_5_count:]
stable_mte_5 = np.mean(top_5_effects)

print(f"Using the average Top 5% value to calculate stable MTE :{stable_mte_5}")
print(f"MTE: {MTE:.4f}")

Using the average Top 5% value to calculate stable MTE :2.0800267465477855


NameError: name 'MTE' is not defined